In [40]:
import pandas as pd
import numpy as np
%matplotlib inline
import config 
import sql_con
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import csv

In [41]:
# get module variables
ROOT_DIR = config.ROOT_DIR
select_records = sql_con.select_records
insert_records = sql_con.insert_records
update_records = sql_con.update_records
conn_odbc = sql_con.conn_odbc
read_contents = sql_con.read_contents

In [42]:
# function to make calls to cryptocompare API
def get_data(url, parameters, headers):
    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [43]:
# function to get api data for top 10 coins
def get_coin_data(coin_list, url, headers):
    coin_data = []
    parameters = { 
        "tsym":"USD",
        "allData":"true"
    }
    for coin in coin_list:
        parameters["fsym"] = coin
        res_json = get_data(url, parameters, headers)
        data = res_json["Data"]
        # iterate through the data and add the coin name to each row
        for row in data:
            row["symbol"] = coin
        coin_data.extend(data)
    return coin_data

In [44]:
# authorization header for making calls to crypto compare API
# read api key from config file using dotev module
headers = {
  "authorization": f"Apikey {config.API_KEY}"
}

In [45]:
# relevant urls for making calls to crypto compare API
top10_url = "https://min-api.cryptocompare.com/data/top/mktcapfull"
hist_url = "https://min-api.cryptocompare.com/data/histoday"

In [46]:
# get top 10 coins by market cap, capture json response
parameters = {
  "tsym":"USD",
  "limit": 10
}

res_json_top10 = get_data(top10_url, parameters, headers)
data_top10 = res_json_top10["Data"]

In [47]:
# capture top 10 coins in a list of dictionaries and write to json file (ingestion layer)
top10_coins = [{"Name": coin["CoinInfo"]["Name"], "FullName": coin["CoinInfo"]["FullName"], "Algorithm": coin["CoinInfo"]["Algorithm"], "ProofType": coin["CoinInfo"]["ProofType"]} for coin in data_top10]
with open(rf"{ROOT_DIR}/data/top10_coins.json", "w") as f:
    f.write(json.dumps(top10_coins))

In [48]:
# make request to cryptocompare api to get historical data for bitcoin quote prices in USD

# parameters = {
#   "fsym": "BTC",
#   "tsym":"USD",
#   "allData":"true"
# }

# res_json = get_data(hist_url, parameters, headers)
# data = res_json["Data"]

In [49]:
import findspark
findspark.init()

In [50]:
# spark session start to begin transforming data (processing layer)
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("crypto_analysis").getOrCreate()

In [51]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType

schema = StructType([
    StructField("Name", StringType(), True),
    StructField("FullName", StringType(), True),
    StructField("Algorithm", StringType(), True),
    StructField("ProofType", StringType(), True)
])
df_top10 = spark.read.option("schema", schema).json(rf"{ROOT_DIR}\data\top10_coins.json").select("Name", "FullName", "Algorithm", "ProofType")
df_top10.show(truncate=False)

+----+------------+---------+---------+
|Name|FullName    |Algorithm|ProofType|
+----+------------+---------+---------+
|BTC |Bitcoin     |SHA-256  |PoW      |
|ETH |Ethereum    |Ethash   |PoS      |
|USDT|Tether      |N/A      |N/A      |
|XRP |XRP         |N/A      |XRP LCP  |
|BNB |Binance Coin|BEP-2    |PoSA     |
|USDC|USD Coin    |N/A      |N/A      |
|ADA |Cardano     |Ouroboros|PoS      |
|DOGE|Dogecoin    |Scrypt   |PoW      |
|ARB |Arbitrum    |N/A      |N/A      |
|APT |Aptos       |N/A      |N/A      |
+----+------------+---------+---------+



In [52]:
coin_list = df_top10.rdd.map(lambda x: x[0]).collect()
coin_list

['BTC', 'ETH', 'USDT', 'XRP', 'BNB', 'USDC', 'ADA', 'DOGE', 'ARB', 'APT']

In [53]:
# read historical data for top 10 coins from cryptocompare API
all_coins_data = get_coin_data(coin_list, hist_url, headers)

In [54]:
# capture data from reponse and write to json file (ingestion layer)
with open(rf"{ROOT_DIR}\data\all_coins_data.json", "w") as f:
    f.write(json.dumps(all_coins_data))

In [55]:
# read ingested json file and print out first 10 records

schema = StructType([
    StructField("symbol", StringType(), True),
    StructField("time", LongType(), True),
    StructField("close", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("open", DoubleType(), True),
    StructField("volumefrom", DoubleType(), True),
    StructField("volumeto", DoubleType(), True),
    StructField("conversionType", StringType(), True),
    StructField("conversionSymbol", StringType(), True)
])

df = spark.read.option("schema", schema).json(rf"{ROOT_DIR}/data/all_coins_data.json")
df.show(n=10)

+-------+----------------+--------------+-------+-------+-------+------+----------+----------+--------+
|  close|conversionSymbol|conversionType|   high|    low|   open|symbol|      time|volumefrom|volumeto|
+-------+----------------+--------------+-------+-------+-------+------+----------+----------+--------+
|0.04951|                |        direct|0.04951|0.04951|0.04951|   BTC|1279324800|      20.0|  0.9902|
|0.08584|                |        direct|0.08585|0.05941|0.04951|   BTC|1279411200|     75.01|   5.092|
| 0.0808|                |        direct|0.09307|0.07723|0.08584|   BTC|1279497600|     574.0|   49.66|
|0.07474|                |        direct|0.08181|0.07426| 0.0808|   BTC|1279584000|     262.0|   20.59|
|0.07921|                |        direct|0.07921|0.06634|0.07474|   BTC|1279670400|     575.0|   42.26|
| 0.0505|                |        direct|0.08181| 0.0505|0.07921|   BTC|1279756800|    2160.0|  129.78|
|0.06262|                |        direct|0.06767| 0.0505| 0.0505

In [56]:
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- conversionType: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time: long (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)



In [57]:
# get ingestion date as current unix epoch time
# write data to csv file after adding ingestion date (csv ingestion point for data pipeline)
from pyspark.sql.functions import unix_timestamp, from_unixtime, col

df = df.withColumn("ingestion_date (unix epoch)", unix_timestamp()).withColumnRenamed("time", "time (unix epoch)")

In [58]:
# change column names for volumes to be more descriptive
df = df.withColumnRenamed("volumefrom", "volume (Crypto Units)").withColumnRenamed("volumeto", "volume (USD)")

In [59]:
df.write.mode("overwrite").option("quote", "\u0000").option("emptyValue", "").csv(rf"{ROOT_DIR}/data/all_coins_data.csv", header=True)

In [60]:
# read from ingested csv file and print out first 10 records
schema = StructType([
    StructField("close", DoubleType(), True),
    StructField("conversionSymbol", StringType(), True),
    StructField("conversionType", StringType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("open", DoubleType(), True),
    StructField("symbol", StringType(), True),
    StructField("time (unix epoch)", LongType(), True),
    StructField("volume (Crypto Units)", DoubleType(), True),
    StructField("volume (USD)", DoubleType(), True),
    StructField("ingestion_date (unix epoch)", LongType(), True)
])

df = spark.read.format("csv").option("header", "true").option("delimiter", ",").schema(schema).load(rf"{ROOT_DIR}/data/all_coins_data.csv")
#df = spark.read.csv(rf"{ROOT_DIR}/data/all_coins_data.csv", header=True)
df.show(n=10,truncate=False)

+-------+----------------+--------------+-------+-------+-------+------+-----------------+---------------------+------------+---------------------------+
|close  |conversionSymbol|conversionType|high   |low    |open   |symbol|time (unix epoch)|volume (Crypto Units)|volume (USD)|ingestion_date (unix epoch)|
+-------+----------------+--------------+-------+-------+-------+------+-----------------+---------------------+------------+---------------------------+
|0.04951|null            |direct        |0.04951|0.04951|0.04951|BTC   |1279324800       |20.0                 |0.9902      |1680648291                 |
|0.08584|null            |direct        |0.08585|0.05941|0.04951|BTC   |1279411200       |75.01                |5.092       |1680648291                 |
|0.0808 |null            |direct        |0.09307|0.07723|0.08584|BTC   |1279497600       |574.0                |49.66       |1680648291                 |
|0.07474|null            |direct        |0.08181|0.07426|0.0808 |BTC   |1279

In [61]:
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- conversionSymbol: string (nullable = true)
 |-- conversionType: string (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- symbol: string (nullable = true)
 |-- time (unix epoch): long (nullable = true)
 |-- volume (Crypto Units): double (nullable = true)
 |-- volume (USD): double (nullable = true)
 |-- ingestion_date (unix epoch): long (nullable = true)



In [62]:
# extract necessary columns
df = df.select(["symbol", "time (unix epoch)", "open", "close", "high", "low", "volume (Crypto Units)", "volume (USD)"])
df.dtypes

[('symbol', 'string'),
 ('time (unix epoch)', 'bigint'),
 ('open', 'double'),
 ('close', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('volume (Crypto Units)', 'double'),
 ('volume (USD)', 'double')]

In [63]:
# get descriptive statistics for numeric columns
df.describe(["open", "close", "high", "low", "volume (Crypto Units)", "volume (USD)"]).show()

+-------+-----------------+-----------------+------------------+-----------------+---------------------+-------------------+
|summary|             open|            close|              high|              low|volume (Crypto Units)|       volume (USD)|
+-------+-----------------+-----------------+------------------+-----------------+---------------------+-------------------+
|  count|            46450|            46450|             46450|            46450|                46450|              46450|
|   mean| 945.719361304651| 946.368930028182| 971.7991685467883|  916.80884632498|   2.46980383910216E7|8.732416636785641E7|
| stddev|5304.725793909023|5306.231182692769|5447.8031052766855|5142.940414625725|  1.599152922706572E8|3.517185911519397E8|
|    min|              0.0|              0.0|               0.0|              0.0|                  0.0|                0.0|
|    max|         67549.14|         67549.14|          68978.64|         66312.42|    1.153458176332E10|  1.112022085477E10|


In [64]:
df.show(n=10)

+------+-----------------+-------+-------+-------+-------+---------------------+------------+
|symbol|time (unix epoch)|   open|  close|   high|    low|volume (Crypto Units)|volume (USD)|
+------+-----------------+-------+-------+-------+-------+---------------------+------------+
|   BTC|       1279324800|0.04951|0.04951|0.04951|0.04951|                 20.0|      0.9902|
|   BTC|       1279411200|0.04951|0.08584|0.08585|0.05941|                75.01|       5.092|
|   BTC|       1279497600|0.08584| 0.0808|0.09307|0.07723|                574.0|       49.66|
|   BTC|       1279584000| 0.0808|0.07474|0.08181|0.07426|                262.0|       20.59|
|   BTC|       1279670400|0.07474|0.07921|0.07921|0.06634|                575.0|       42.26|
|   BTC|       1279756800|0.07921| 0.0505|0.08181| 0.0505|               2160.0|      129.78|
|   BTC|       1279843200| 0.0505|0.06262|0.06767| 0.0505|               2402.5|      141.07|
|   BTC|       1279929600|0.06262|0.05454|0.06161|0.05049|  

In [65]:
# set spark session timezone to UTC to have a uniform reference point for all date related fields
spark.conf.set("spark.sql.session.timeZone", "UTC")
df = df.withColumn("date_time (unix)", from_unixtime("time (unix epoch)", "yyyy-MM-dd HH:mm:ss"))
spark.conf.unset("spark.sql.session.timeZone")

In [66]:
# timezone will default to system timezone (Easter Standard Time) in absence of specific spark.sql.session.timeZone setting
df.withColumn("date_time", from_unixtime("time (unix epoch)", "yyyy-MM-dd HH:mm:ss")).show(n=10, truncate=False)

+------+-----------------+-------+-------+-------+-------+---------------------+------------+-------------------+-------------------+
|symbol|time (unix epoch)|open   |close  |high   |low    |volume (Crypto Units)|volume (USD)|date_time (unix)   |date_time          |
+------+-----------------+-------+-------+-------+-------+---------------------+------------+-------------------+-------------------+
|BTC   |1279324800       |0.04951|0.04951|0.04951|0.04951|20.0                 |0.9902      |2010-07-17 00:00:00|2010-07-16 20:00:00|
|BTC   |1279411200       |0.04951|0.08584|0.08585|0.05941|75.01                |5.092       |2010-07-18 00:00:00|2010-07-17 20:00:00|
|BTC   |1279497600       |0.08584|0.0808 |0.09307|0.07723|574.0                |49.66       |2010-07-19 00:00:00|2010-07-18 20:00:00|
|BTC   |1279584000       |0.0808 |0.07474|0.08181|0.07426|262.0                |20.59       |2010-07-20 00:00:00|2010-07-19 20:00:00|
|BTC   |1279670400       |0.07474|0.07921|0.07921|0.06634|575.

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume(USD) of stock traded for a day

In [67]:
df2 = df.withColumn("HV Ratio", col("high")/col("volume (USD)"))

# display dataframe in descending order of HV Ratio
df2.sort("HV Ratio", ascending=False).show(truncate=False)

+------+-----------------+------+------+------+------+---------------------+------------+-------------------+------------------+
|symbol|time (unix epoch)|open  |close |high  |low   |volume (Crypto Units)|volume (USD)|date_time (unix)   |HV Ratio          |
+------+-----------------+------+------+------+------+---------------------+------------+-------------------+------------------+
|USDT  |1441152000       |0.96  |1.15  |1.15  |0.96  |1.252E-4             |1.44E-4     |2015-09-02 00:00:00|7986.11111111111  |
|USDT  |1424476800       |1.15  |1.5   |1.5   |1.15  |1.415E-4             |2.122E-4    |2015-02-21 00:00:00|7068.80301602262  |
|USDT  |1456185600       |1.1   |1.1   |1.1   |1.1   |1.518E-4             |1.67E-4     |2016-02-23 00:00:00|6586.82634730539  |
|USDT  |1420761600       |1.0   |1.0   |1.0   |1.0   |2.4E-4               |2.4E-4      |2015-01-09 00:00:00|4166.666666666667 |
|USDT  |1422662400       |0.9274|4.37  |4.37  |0.874 |0.005                |0.02185     |2015-01-

In [68]:
# sort by date_time (unix) in descending order to get HV Ratiio for most recent dates
df2.sort("date_time (unix)", ascending=False).show(truncate=False)

+------+-----------------+--------+--------+--------+--------+---------------------+--------------+-------------------+---------------------+
|symbol|time (unix epoch)|open    |close   |high    |low     |volume (Crypto Units)|volume (USD)  |date_time (unix)   |HV Ratio             |
+------+-----------------+--------+--------+--------+--------+---------------------+--------------+-------------------+---------------------+
|BTC   |1680566400       |27810.08|28206.13|28437.75|27674.33|32650.85             |9.1800723453E8|2023-04-04 00:00:00|3.097769704893395E-5 |
|XRP   |1680566400       |0.4964  |0.5044  |0.5073  |0.4871  |7.91145846E7         |3.93053385E7  |2023-04-04 00:00:00|1.2906643712024003E-8|
|BNB   |1680566400       |308.66  |310.95  |312.45  |308.26  |13539.82             |4187276.49    |2023-04-04 00:00:00|7.461890819633932E-5 |
|USDT  |1680566400       |1.0     |1.0     |1.002   |0.9994  |2.4393991638E8       |2.4397732647E8|2023-04-04 00:00:00|4.106939011495432E-9 |
|ADA  

In [69]:
df2.printSchema()

root
 |-- symbol: string (nullable = true)
 |-- time (unix epoch): long (nullable = true)
 |-- open: double (nullable = true)
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- volume (Crypto Units): double (nullable = true)
 |-- volume (USD): double (nullable = true)
 |-- date_time (unix): string (nullable = true)
 |-- HV Ratio: double (nullable = true)



In [70]:
df2.groupby(col("symbol").alias("sym")).max("high").show()

+----+---------+
| sym|max(high)|
+----+---------+
| ARB|    1.266|
| ETH|  4865.94|
|DOGE|   0.7398|
|USDC|    1.643|
| BTC| 68978.64|
|USDT|   207.09|
| XRP|     3.29|
| BNB|   693.73|
| APT|    20.33|
| ADA|    3.097|
+----+---------+



#### What day had the Peak High in Price for each cryptocurrency in dataset?

In [71]:
df_max_high = df2.groupby(col("symbol").alias("sym")).agg({"high": "max"})
df2.join(df_max_high, (df2["symbol"] == df_max_high["sym"]) & (df2["high"] == df_max_high["max(high)"])).select(["symbol", "date_time (unix)", "high"]).show()

+------+-------------------+--------+
|symbol|   date_time (unix)|    high|
+------+-------------------+--------+
|   BTC|2021-11-10 00:00:00|68978.64|
|   ETH|2021-11-10 00:00:00| 4865.94|
|  USDT|2015-11-06 00:00:00|  207.09|
|   XRP|2018-01-04 00:00:00|    3.29|
|   BNB|2021-11-08 00:00:00|  693.73|
|  USDC|2020-03-12 00:00:00|   1.643|
|   ADA|2021-09-02 00:00:00|   3.097|
|  DOGE|2021-05-08 00:00:00|  0.7398|
|   ARB|2023-04-04 00:00:00|   1.266|
|   APT|2023-01-30 00:00:00|   20.33|
+------+-------------------+--------+



#### What is the mean closing price for each cryptocurrency?

In [72]:
df2.groupby("symbol").mean("close").sort("avg(close)", ascending=False).show()

+------+--------------------+
|symbol|          avg(close)|
+------+--------------------+
|   BTC|   8894.902664497304|
|   ETH|   501.3238324865445|
|   BNB|    65.5657388159311|
|  USDT|   0.759260495156095|
|  USDC| 0.35313466092572676|
|   APT| 0.32361011840688925|
|   XRP|  0.2289785164693219|
|   ADA| 0.20545459203444555|
|  DOGE|0.026359360598492997|
|   ARB|2.667384284176534...|
+------+--------------------+



#### What is min and max volume (both unit assets and price) for each cryptocurrency?

In [73]:
from pyspark.sql.functions import max, min
df2.groupby("symbol").agg(
    max("volume (Crypto Units)").alias("max_volume (Crypto Units)"),
    min("volume (Crypto Units)").alias("min_volume (Crypto Units)"),
    max("volume (USD)").alias("max_volume (USD)"),
    min("volume (USD)").alias("min_volume (USD)")
).show()

+------+-------------------------+-------------------------+-----------------+----------------+
|symbol|max_volume (Crypto Units)|min_volume (Crypto Units)| max_volume (USD)|min_volume (USD)|
+------+-------------------------+-------------------------+-----------------+----------------+
|   ARB|               5369069.89|                      0.0|       6593005.93|             0.0|
|   ETH|            1.098033209E7|                      0.0|  9.65796855596E9|             0.0|
|  DOGE|        1.153458176332E10|                      0.0|  2.12237452234E9|             0.0|
|  USDC|           5.6439868532E8|                      0.0|    5.333564084E8|             0.0|
|   BTC|                572349.32|                      0.0|1.112022085477E10|             0.0|
|  USDT|          4.23247075921E9|                      0.0|  4.18769644556E9|             0.0|
|   XRP|          3.61211042715E9|                      0.0|  1.56522873526E9|             0.0|
|   BNB|            1.082578296E7|      

#### What is the Pearson's correlation coefficient between high and volume (USD) for each cryptocurrency?

In [74]:
from pyspark.sql.functions import corr
df2.groupby("symbol").agg(corr("high", "volume (USD)")).show()

+------+------------------------+
|symbol|corr(high, volume (USD))|
+------+------------------------+
|   ARB|                     1.0|
|   ETH|      0.7296758368697659|
|  DOGE|      0.7214272997816695|
|  USDC|      0.1292183119594394|
|   BTC|       0.777549881086647|
|  USDT|    0.002495057225778...|
|   XRP|      0.6677528906152208|
|   BNB|      0.4592640013897647|
|   APT|       0.861090128365362|
|   ADA|       0.721346715928943|
+------+------------------------+



#### What is max metrics per year for each cryptocurrency?

In [75]:
from pyspark.sql.functions import year
max_metrics_df = df2.withColumn("year", year(df["date_time (unix)"])).groupby(["symbol", "year"]).agg(
                                                max("high").alias("max_high"),
                                                max("close").alias("max_close"),
                                                max("open").alias("max_open"),
                                                max("low").alias("max_low"),
                                                max("volume (USD)").alias("max_volume (USD)"))
max_metrics_df.sort("max_high", ascending=False).show(truncate=False)

+------+----+--------+---------+--------+--------+-----------------+
|symbol|year|max_high|max_close|max_open|max_low |max_volume (USD) |
+------+----+--------+---------+--------+--------+-----------------+
|BTC   |2021|68978.64|67549.14 |67549.14|66312.42|1.112022085477E10|
|BTC   |2022|48208.48|47737.35 |47737.35|47018.87|4.50136816633E9  |
|BTC   |2020|29301.67|28972.4  |28889.61|27957.13|3.19465446793E9  |
|BTC   |2023|29172.9 |28477.29 |28477.29|28265.42|2.74410290012E9  |
|BTC   |2017|19870.62|19345.49 |19346.6 |18750.91|6.2457315084E9   |
|BTC   |2018|17252.85|17172.3  |17174.5 |16286.57|3.84265174145E9  |
|BTC   |2019|13826.76|12913.28 |12913.28|12117.31|2.6858723651E9   |
|ETH   |2021|4865.94 |4810.97  |4810.97 |4710.0  |9.65796855596E9  |
|ETH   |2022|3891.61 |3830.92  |3830.92 |3718.46 |4.10783640823E9  |
|ETH   |2023|1889.35 |1868.06  |1822.21 |1811.11 |1.18646114136E9  |
|ETH   |2018|1422.86 |1385.02  |1385.02 |1271.07 |2.03401088899E9  |
|BTC   |2013|1241.92 |1237.55  |12

#### What is average closing price for each month for each cryptocurrency?

In [76]:
from pyspark.sql.functions import month
avg_metrics_month = df2.withColumn("month", month(df["date_time (unix)"])).groupby(["symbol", "month"]).mean("close")
avg_metrics_month.sort("avg(close)", ascending=False).show()

+------+-----+------------------+
|symbol|month|        avg(close)|
+------+-----+------------------+
|   BTC|    3| 11095.00185012407|
|   BTC|    2|10355.330763215256|
|   BTC|    4|10317.462448351647|
|   BTC|    1|  9511.98708263027|
|   BTC|   11| 9116.573878461542|
|   BTC|   12| 8976.780759057074|
|   BTC|    5| 8860.711142473117|
|   BTC|   10| 8548.841935334987|
|   BTC|    8| 7877.584074367245|
|   BTC|    9| 7562.788398384617|
|   BTC|    6| 7510.352950000002|
|   BTC|    7| 7012.631505452193|
|   ETH|    2| 568.6205694822889|
|   ETH|    3| 557.9127791563276|
|   ETH|    5|  547.094997311828|
|   ETH|    1| 546.9078258064515|
|   ETH|    4| 539.8541373626374|
|   ETH|   11| 533.5815920512822|
|   ETH|   12|  522.392269975186|
|   ETH|   10| 482.2133401985114|
+------+-----+------------------+
only showing top 20 rows

